## Camada Silver – Tratamento e Padronização

Este notebook aplica regras de limpeza, conversão de tipos e padronização textual aos dados da CEAP, mantendo a granularidade original.



**Objetivo da camada Silver:** 
Transformar os dados brutos da camada Bronze em dados confiáveis, tipados, padronizados e consistentes, mantendo a granularidade original, sem ainda aplicar modelagem analítica.

**Visão geral do mapeamento**

| Ação       | Bronze    | Silver                   |
| ---------- | --------- | ------------------------ |
| Estrutura  | CSV cru   | Tabela Spark estruturada |
| Tipos      | String    | Tipos corretos           |
| Datas      | Texto ISO | Date                     |
| Valores    | String    | Decimal                  |
| Duplicatas | Mantidas  | Avaliadas                |
| Outliers   | Mantidos  | Identificados            |
| Semântica  | Original  | Preservada               |


Na camada Silver foram aplicadas transformações de padronização de tipos, normalização textual, conversão de datas e valores monetários, além da avaliação de valores nulos e inconsistentes. Não foram realizadas agregações ou alterações de granularidade, mantendo-se a correspondência de uma linha por despesa.


**Tabela Silver proposta:**

SILVER_DESPESA_CEAP

####Mapeamento campo a campo:

**Parlamentar / Identificação pessoal**
| Campo Bronze            | Campo Silver               | Tipo Silver | Regra / Observação                                     |
| ----------------------- | -------------------------- | ----------- | ------------------------------------------------------ |
| `txNomeParlamentar`     | `nome_parlamentar`         | string      | `trim()`                                               |
| `cpf`                   | `cpf`                      | string      | manter como string; pode mascarar/ter zeros à esquerda |
| `ideCadastro`           | `id_cadastro`              | int         | `cast(int)` (identificador interno histórico)          |
| `nuCarteiraParlamentar` | `num_carteira_parlamentar` | int         | `cast(int)`                                            |
| `nuLegislatura`         | `num_legislatura`          | int         | `cast(int)`                                            |
| `codLegislatura`        | `cod_legislatura`          | int         | `cast(int)`                                            |
| `sgUF`                  | `sigla_uf_parlamentar`     | string      | `trim()+upper()`                                       |
| `sgPartido`             | `sigla_partido`            | string      | `trim()+upper()`                                       |
| `nuDeputadoId`          | `id_parlamentar`           | int         | `cast(int)` (**FK futura para dimensão**)              |

**Identificação do documento / despesa**
| Campo Bronze (CSV) | Campo Silver       | Tipo Silver | Regra / Observação                   |
| ------------------ | ------------------ | ----------- | ------------------------------------ |
| `ideDocumento`     | `id_documento`     | string      | `trim()`                             |
| `txtNumero`        | `numero_documento` | string      | `trim()` (**não é único**)           |
| `indTipoDocumento` | `tipo_documento`   | string/int  | manter (normalizar depois se quiser) |
| `urlDocumento`     | `url_documento`    | string      | manter                               |

**Classificação da despesa (subcota / tipo)**
| Campo Bronze (CSV)          | Campo Silver                   | Tipo Silver | Regra / Observação                       |
| --------------------------- | ------------------------------ | ----------- | ---------------------------------------- |
| `numSubCota`                | `id_tipo_despesa`              | int         | `cast(int)` (equivale ao “tipo”/subcota) |
| `txtDescricao`              | `descricao_tipo_despesa`       | string      | `trim()`                                 |
| `numEspecificacaoSubCota`   | `id_especificacao_tipo`        | int         | `cast(int)`                              |
| `txtDescricaoEspecificacao` | `descricao_especificacao_tipo` | string      | `trim()`                                 |

**Fornecedor**
| Campo Bronze (CSV) | Campo Silver         | Tipo Silver | Regra / Observação  |
| ------------------ | -------------------- | ----------- | ------------------- |
| `txtFornecedor`    | `nome_fornecedor`    | string      | `trim()`            |
| `txtCNPJCPF`       | `cnpj_cpf_formatado` | string      | `trim()` (como vem) |

**Datas e Valores**
| Campo Bronze (CSV) | Campo Silver      | Tipo Silver   | Regra / Observação      |
| ------------------ | ----------------- | ------------- | ----------------------- |
| `datEmissao`       | `data_emissao`    | date          | `to_date()` (parse ISO) |
| `vlrDocumento`     | `valor_documento` | decimal(12,2) | `cast(decimal)`         |
| `vlrGlosa`         | `valor_glosa`     | decimal(12,2) | `cast(decimal)`         |
| `vlrLiquido`       | `valor_liquido`   | decimal(12,2) | `cast(decimal)`         |
| `numMes`           | `mes_ref`         | int           | `cast(int)` (auxiliar)  |
| `numAno`           | `ano_ref`         | int           | `cast(int)` (auxiliar)  |

**Parcelamento / Lotes / Ressarcimento**
| Campo Bronze (CSV)        | Campo Silver                 | Tipo Silver   | Regra / Observação                   |
| ------------------------- | ---------------------------- | ------------- | ------------------------------------ |
| `numParcela`              | `num_parcela`                | int           | `cast(int)`                          |
| `numLote`                 | `num_lote`                   | int           | `cast(int)`                          |
| `numRessarcimento`        | `num_ressarcimento`          | string/int    | manter (alguns datasets têm alfanum) |
| `datPagamentoRestituicao` | `data_pagamento_restituicao` | date          | `to_date()`                          |
| `vlrRestituicao`          | `valor_restituicao`          | decimal(12,2) | `cast(decimal)`                      |

**Itens de Passagem (quando aplicável)**
| Campo Bronze (CSV) | Campo Silver | Tipo Silver | Regra / Observação |
| ------------------ | ------------ | ----------- | ------------------ |
| `txtPassageiro`    | `passageiro` | string      | `trim()`           |
| `txtTrecho`        | `trecho`     | string      | `trim()`           |


**Identificação lógica da despesa**
(id_documento, id_parlamentar, data_emissao, valor_documento)
---

# Transformações Técnicas

In [0]:
spark.sql("USE CATALOG mvp_ed_ceap")
spark.sql("USE SCHEMA layer_silver")

DataFrame[]

In [0]:
from pyspark.sql.functions import (
    col, trim, upper, to_date, regexp_replace
)
from pyspark.sql.types import DecimalType

df_bronze = spark.table("mvp_ed_ceap.layer_bronze.bronze_ceap_despesas")

In [0]:
# Renomeação de colunas (padronização semântica)
df = (
    df_bronze
    # Parlamentar
    .withColumnRenamed("txNomeParlamentar", "nome_parlamentar")
    .withColumnRenamed("cpf", "cpf")
    .withColumnRenamed("ideCadastro", "id_cadastro")
    .withColumnRenamed("nuCarteiraParlamentar", "num_carteira_parlamentar")
    .withColumnRenamed("nuLegislatura", "num_legislatura")
    .withColumnRenamed("codLegislatura", "cod_legislatura")
    .withColumnRenamed("sgUF", "sigla_uf_parlamentar")
    .withColumnRenamed("sgPartido", "sigla_partido")
    .withColumnRenamed("nuDeputadoId", "id_parlamentar")

    # Documento / despesa
    .withColumnRenamed("ideDocumento", "id_documento")
    .withColumnRenamed("txtNumero", "numero_documento")
    .withColumnRenamed("indTipoDocumento", "tipo_documento")
    .withColumnRenamed("urlDocumento", "url_documento")

    # Classificação
    .withColumnRenamed("numSubCota", "id_tipo_despesa")
    .withColumnRenamed("txtDescricao", "descricao_tipo_despesa")
    .withColumnRenamed("numEspecificacaoSubCota", "id_especificacao_tipo")
    .withColumnRenamed("txtDescricaoEspecificacao", "descricao_especificacao_tipo")

    # Fornecedor
    .withColumnRenamed("txtFornecedor", "nome_fornecedor")
    .withColumnRenamed("txtCNPJCPF", "cnpj_cpf_formatado")

    # Datas e valores
    .withColumnRenamed("datEmissao", "data_emissao_raw")
    .withColumnRenamed("vlrDocumento", "valor_documento_raw")
    .withColumnRenamed("vlrGlosa", "valor_glosa_raw")
    .withColumnRenamed("vlrLiquido", "valor_liquido_raw")
    .withColumnRenamed("numMes", "mes_ref")
    .withColumnRenamed("numAno", "ano_ref")

    # Parcelamento / ressarcimento
    .withColumnRenamed("numParcela", "num_parcela")
    .withColumnRenamed("numLote", "num_lote")
    .withColumnRenamed("numRessarcimento", "num_ressarcimento")
    .withColumnRenamed("datPagamentoRestituicao", "data_pagamento_restituicao_raw")
    .withColumnRenamed("vlrRestituicao", "valor_restituicao_raw")

    # Passagens
    .withColumnRenamed("txtPassageiro", "passageiro")
    .withColumnRenamed("txtTrecho", "trecho")
)


In [0]:
# Padronização textual (trim / upper)
df = (
    df
    .withColumn("nome_parlamentar", trim(col("nome_parlamentar")))
    .withColumn("sigla_partido", upper(trim(col("sigla_partido"))))
    .withColumn("sigla_uf_parlamentar", upper(trim(col("sigla_uf_parlamentar"))))
    .withColumn("descricao_tipo_despesa", trim(col("descricao_tipo_despesa")))
    .withColumn("descricao_especificacao_tipo", trim(col("descricao_especificacao_tipo")))
    .withColumn("nome_fornecedor", trim(col("nome_fornecedor")))
    .withColumn("numero_documento", trim(col("numero_documento")))
)


In [0]:
# Conversão de Datas (ISO -> date)
df = (
    df
    .withColumn("data_emissao", to_date(col("data_emissao_raw")))
    .withColumn(
        "data_pagamento_restituicao",
        to_date(col("data_pagamento_restituicao_raw"))
    )
)

In [0]:
# Conversão de valores monetários (string → decimal)
df = (
    df
    .withColumn(
        "valor_documento",
        col("valor_documento_raw").cast(DecimalType(12, 2))
    )
    .withColumn(
        "valor_glosa",
        col("valor_glosa_raw").cast(DecimalType(12, 2))
    )
    .withColumn(
        "valor_liquido",
        col("valor_liquido_raw").cast(DecimalType(12, 2))
    )
    .withColumn(
        "valor_restituicao",
        col("valor_restituicao_raw").cast(DecimalType(12, 2))
    )
)

In [0]:
# Casts finais de tipos numéricos
df = (
    df
    .withColumn("id_parlamentar", col("id_parlamentar").cast("int"))
    .withColumn("id_cadastro", col("id_cadastro").cast("int"))
    .withColumn("num_carteira_parlamentar", col("num_carteira_parlamentar").cast("int"))
    .withColumn("num_legislatura", col("num_legislatura").cast("int"))
    .withColumn("cod_legislatura", col("cod_legislatura").cast("int"))
    .withColumn("id_tipo_despesa", col("id_tipo_despesa").cast("int"))
    .withColumn("id_especificacao_tipo", col("id_especificacao_tipo").cast("int"))
    .withColumn("mes_ref", col("mes_ref").cast("int"))
    .withColumn("ano_ref", col("ano_ref").cast("int"))
    .withColumn("num_parcela", col("num_parcela").cast("int"))
    .withColumn("num_lote", col("num_lote").cast("int"))
)


In [0]:
# CNPJ/CPF somente dígitos
df = df.withColumn(
    "cnpj_cpf_digits",
    regexp_replace(col("cnpj_cpf_formatado"), r"[^0-9]", "")
)


In [0]:
# Seleção final (remove colunas raw)
df_silver = df.drop(
    "data_emissao_raw",
    "valor_documento_raw",
    "valor_glosa_raw",
    "valor_liquido_raw",
    "data_pagamento_restituicao_raw",
    "valor_restituicao_raw"
)


In [0]:
(
    df_silver
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_ceap_despesas")
)

---
# Exploração e Análise de Qualidade

**Visão geral do dataset (sanity check)**

In [0]:
%sql
SELECT
  COUNT(*) AS total_registros,
  COUNT(DISTINCT id_parlamentar) AS qtd_parlamentares,
  MIN(data_emissao) AS data_min,
  MAX(data_emissao) AS data_max
FROM silver_ceap_despesas;


total_registros,qtd_parlamentares,data_min,data_max
231711,594,2002-06-25,2025-03-27


**Completude (Valores Nulos)**

In [0]:
%sql

--Campos críticos
SELECT
  SUM(CASE WHEN id_parlamentar IS NULL THEN 1 ELSE 0 END) AS id_parlamentar_nulos,
  SUM(CASE WHEN data_emissao IS NULL THEN 1 ELSE 0 END) AS data_emissao_nulos,
  SUM(CASE WHEN valor_documento IS NULL THEN 1 ELSE 0 END) AS valor_documento_nulos
FROM silver_ceap_despesas;

id_parlamentar_nulos,data_emissao_nulos,valor_documento_nulos
8,9459,4


In [0]:
%sql

-- Campos opcionais (espera-se nulos)
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN nome_fornecedor IS NULL THEN 1 ELSE 0 END) AS fornecedor_nulo,
  SUM(CASE WHEN cnpj_cpf_formatado IS NULL THEN 1 ELSE 0 END) AS cnpj_nulo,
  SUM(CASE WHEN passageiro IS NULL THEN 1 ELSE 0 END) AS passageiro_nulo
FROM silver_ceap_despesas;


total,fornecedor_nulo,cnpj_nulo,passageiro_nulo
231711,4,57014,173783


**Consistência de valores monetários**

In [0]:
%sql

--Valores negativos ou inválidos
SELECT
  COUNT(*) AS valores_negativos
FROM silver_ceap_despesas
WHERE valor_documento < 0
   OR valor_glosa < 0
   OR valor_liquido < 0;


valores_negativos
10593


In [0]:
%sql

--Relação entre valores
SELECT
  COUNT(*) AS inconsistencias
FROM silver_ceap_despesas
WHERE valor_liquido > valor_documento;


inconsistencias
0


**Distribuição e outliers (exploração, não remoção)**

In [0]:
%sql

--Estastisticas básicas
SELECT
  MIN(valor_documento) AS min_valor,
  PERCENTILE_APPROX(valor_documento, 0.5) AS mediana,
  AVG(valor_documento) AS media,
  MAX(valor_documento) AS max_valor
FROM silver_ceap_despesas;


min_valor,mediana,media,max_valor
-4193.99,250.00,1097.033420,209948.00


**Duplicidade lógica**

In [0]:
%sql
SELECT
  id_documento,
  id_parlamentar,
  data_emissao,
  COUNT(*) AS qtd
FROM silver_ceap_despesas
GROUP BY id_documento, id_parlamentar, data_emissao
HAVING COUNT(*) > 1
ORDER BY qtd DESC;

id_documento,id_parlamentar,data_emissao,qtd
0,2973,null,47
0,1987,null,45
0,3040,null,44
0,1071,null,44
0,3665,null,44
0,3326,null,43
0,974,null,40
0,1531,null,39
0,3747,null,39
0,1097,null,37


**Consistência temporal**

In [0]:
%sql
SELECT
  COUNT(*) AS datas_fora_do_ano
FROM silver_ceap_despesas
WHERE ano_ref <> YEAR(data_emissao);


datas_fora_do_ano
1073


**Domínios categóricos**

In [0]:
%sql

--Tipos de despesas
SELECT
  descricao_tipo_despesa,
  COUNT(*) AS qtd
FROM silver_ceap_despesas
GROUP BY descricao_tipo_despesa
ORDER BY qtd DESC;


descricao_tipo_despesa,qtd
COMBUSTÍVEIS E LUBRIFICANTES.,72728
PASSAGEM AÉREA - SIGEPA,57010
"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",21432
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,20586
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,16425
TELEFONIA,15600
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,10133
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,7494
"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.",7486
PASSAGEM AÉREA - RPA,834


In [0]:
%sql

--UF do parlamentar
SELECT
  sigla_uf_parlamentar,
  COUNT(*) AS qtd
FROM silver_ceap_despesas
GROUP BY sigla_uf_parlamentar
ORDER BY qtd DESC;


sigla_uf_parlamentar,qtd
SP,30398
MG,29571
RS,25635
RJ,19281
BA,17388
PR,16713
PE,12117
SC,9920
CE,7930
MA,5021


**Resumo da qualidade**

In [0]:
%sql
SELECT
  COUNT(*) AS total_registros,
  SUM(CASE WHEN valor_documento IS NULL THEN 1 ELSE 0 END) AS valor_nulo,
  SUM(CASE WHEN valor_documento = 0 THEN 1 ELSE 0 END) AS valor_zero,
  SUM(CASE WHEN data_emissao IS NULL THEN 1 ELSE 0 END) AS data_nula
FROM silver_ceap_despesas;

total_registros,valor_nulo,valor_zero,data_nula
231711,4,0,9459
